In [1]:
import ollama
import re
import os

In [2]:
def make_prompt(transcript_text):
    ln_1 = f'\nThe following text is a transcript from a youtube video: {transcript_text}'
    ln_3 = "\n\n\n List out all company names mentioned in the transcript."
    ln_4 = "\n Put the industry of the company identified beside the company name."
    ln_5 = "\n The transcript enclosed might contain questions, do not answer those questions."
    ln_6 = "\n Maximise the list!!!"
    content = ln_1 + ln_3 + ln_4 + ln_5 + ln_6
    return content

def make_messages_llm(prompt, transcript_text):
    messages = [
        {'role': 'system', 'content': 'You are a state of the art Named Entity Recognition model.'},
        {'role': 'user', 'content': f"\n\n{prompt}"},
    ]
    return messages

def send_request(messages):
    options = {
        #'temperature': 1.5, # very creative
        #'temperature': 0 # very conservative (good for coding and correct syntax)
        'temperature': 1.5
    }
    response = ollama.chat(model=model, messages=messages, options=options)
    answer = response['message']['content']
    return answer
 
def clean_transcript_text(text):
    text =  text.replace("\\n", ' ')
    text = re.sub('\[.+?\]', ' ', text)
    text = re.sub('\(.+?\)', ' ', text)
    text = text.replace("\n",' ')
    return text

def process_response_1(text):
    text = text.replace("**", "")
    texts = text.split("\n")
    texts = [text for text in texts if not ' none' in text.lower()]
    texts = [text for text in texts if not '(unknown' in text.lower()]
    lns_with_company_name  = [text for text in texts if re.search('(^[\d]+\.)|(^-{1}.+)|(^\*)', text.strip())]
    lns_with_company_name = [re.sub("^[\d]+\.", '', text.strip()) for text in lns_with_company_name]
    lns_with_company_name = [re.sub("^\*", '', text.strip()) for text in lns_with_company_name]
    lns_with_company_name = [re.sub("^-", '', text.strip()) for text in lns_with_company_name]
    lns_with_company_name = [text.strip() for text in lns_with_company_name]
    return lns_with_company_name

In [3]:
vid_id = "oqn1ZD8QUm4"
transcript_dir = r"E:\youtube_data\channel_scraper\wsj\transcripts"

with open(os.path.join(transcript_dir, f"{vid_id}.txt"), 'r') as f:
    content = f.read()
    transcript_text = clean_transcript_text(content)

In [4]:
transcript_text

"The US government seized First Republic Bank on May 1st and struck a deal to sell the  bulk of its operations to JP Morgan regulators hope the move will head off  a chaotic collapse that could reignite the banking crisis. Ben Eisen covers banking and finance for the journal. He explains how first Republic became the second largest bank to fail in the US and what it  means for customers investors and the industry. First Republic had this business model that was just sort of been perfectly built for a certain  time period when interest rates were really low. Um The bank collected a lot of deposits that made a lot of mortgages but when rates suddenly shifted. Last year, the bank's business model started to look a little bit more precarious after  Silicon Valley Bank and signature failed. In March, everyone pretty much immediately started looking to first Republic as the next risky bank in the  system. Silicon Valley Bank and First Republic, they had similar enough client bases and simila

In [5]:
model = "gemma" # 'llama3' # qwen:7b #gemma
messages = make_messages_llm(make_prompt(transcript_text), transcript_text)

response = send_request(messages)
print(response)

- **First Republic Bank** (Banking)
- **JP Morgan** (Investment banking)
- **Silicon Valley Bank** (Banking)
- **Signature Bank** (Banking)
- **Federal Deposit Insurance Corp (FDIC)** (Financial regulator)


In [8]:
print(process_response_1(response))

['First Republic Bank (Banking)', 'JP Morgan (Investment Banking)', 'Silicon Valley Bank (Banking)', 'Signature Bank (Banking)']
